# Analysis of practice level prescribing of triptan drugs

[English Prescribing Dataset](https://opendata.nhsbsa.net/dataset/english-prescribing-data-epd)

[Drug Tariff Nov 2020](https://www.nhsbsa.nhs.uk/sites/default/files/2020-10/Drug%20Tariff%20November%202020.pdf)

[API call in R](https://gitlab.com/nhsbsa/insight/open-data-portal-api/-/blob/master/OpenDataAPIQuery.R)

**NIC (from NHSBSA website)**: "The amount that would be paid using the basic price of the prescribed drug or appliance and the quantity prescribed. Sometimes called the 'Net Ingredient Cost' (NIC). **The basic price is given either in the Drug Tariff or is determined from prices published by manufacturers, wholesalers or suppliers.** Basic price is set out in Parts 8 and 9 of the Drug Tariff. **For any drugs or appliances not in Part 8, the price is usually taken from the manufacturer, wholesaler or supplier of the product.**"

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium
%matplotlib inline
sns.set(color_codes=True)

In [2]:
pd.set_option('display.max_columns', 11)

In [3]:
df = pd.read_csv("./data/EPD_202008.csv", low_memory = False)

In [4]:
df.drop(["REGIONAL_OFFICE_NAME", "REGIONAL_OFFICE_CODE", 'STP_NAME', 'STP_CODE', 'PCO_NAME', 'PCO_CODE', 'PRACTICE_NAME','ADDRESS_1', 'ADDRESS_2', 'ADDRESS_3', 'ADDRESS_4', 'ADQUSAGE', 'POSTCODE', 'BNF_CODE', 'BNF_CHAPTER_PLUS_CODE', 'BNF_CODE', 'BNF_CHAPTER_PLUS_CODE', 'BNF_CHEMICAL_SUBSTANCE'], axis = 1, inplace = True)

In [5]:
df.columns = [x.lower() for x in df.columns]

In [6]:
df.rename(columns = {"chemical_substance_bnf_descr":"drug" , "bnf_description":"drug_desc"}, inplace = True)

In [7]:
df['normalised_nic'] = df['nic']/df['total_quantity']

In [8]:
df['normalised_nic'] = df['normalised_nic'].round(2)

In [10]:
triptan_df = df[df["drug"].str.contains("triptan",na=False)]
triptan_df = triptan_df[~triptan_df["drug"].str.contains("Oxitriptan")]

**88336 prescriptions for triptan drugs across England in August 2020**

- 47% Sumatriptan
- 20% Zolmitriptan
- 19% Rizatriptan
- 7.2% Naratriptan
- 3.4% Frovatriptan
- 2.4% Almotriptan
- 1.0% Eletriptan

Total cost of triptan prescriptions in the month:

In [11]:
triptan_df["drug"].value_counts(normalize=True)

Sumatriptan succinate        0.470624
Zolmitriptan                 0.201435
Rizatriptan                  0.187387
Naratriptan hydrochloride    0.071930
Frovatriptan                 0.034369
Almotriptan                  0.024135
Eletriptan                   0.010120
Name: drug, dtype: float64

In [12]:
triptan_grp = triptan_df.groupby(["drug"])

In [13]:
triptan_grp.get_group("Rizatriptan")

,year_month,practice_code,drug,drug_desc,quantity,items,total_quantity,nic,actual_cost,unidentified,normalised_nic
1255,202008,L83089,Rizatriptan,Rizatriptan 10mg tablets,12.0,1,12.0,15.08,14.03370,N,1.26
1256,202008,L83089,Rizatriptan,Rizatriptan 10mg tablets,6.0,3,18.0,22.62,21.06915,N,1.26
1257,202008,L83089,Rizatriptan,Rizatriptan 10mg oral lyophilisates sugar free,6.0,2,12.0,53.48,49.75020,N,4.46
1258,202008,L83089,Rizatriptan,Rizatriptan 10mg orodispersible tablets sugar ...,6.0,1,6.0,10.40,9.68226,N,1.73
1259,202008,L83089,Rizatriptan,Rizatriptan 10mg orodispersible tablets sugar ...,12.0,1,12.0,20.80,19.35212,N,1.73
...,...,...,...,...,...,...,...,...,...,...,...
16676507,202008,H85020,Rizatriptan,Rizatriptan 10mg tablets,12.0,2,24.0,30.16,28.06740,N,1.26
16676508,202008,H85020,Rizatriptan,Rizatriptan 10mg tablets,6.0,1,6.0,7.54,7.02305,N,1.26
16676509,202008,H85020,Rizatriptan,Rizatriptan 10mg oral lyophilisates sugar free,6.0,1,6.0,26.74,24.87510,N,4.46
16676510,202008,H85020,Rizatriptan,Rizatriptan 10mg orodispersible tablets sugar ...,9.0,1,9.0,15.60,14.51719,N,1.73


**When triptan drug prescriptions are analysed per drug, variation in the prescription of branded/generic versions can be seen**

In [14]:
triptan_grp['drug_desc'].value_counts(normalize=True)

drug                       drug_desc                                                  
Almotriptan                Almotriptan 12.5mg tablets                                     0.983583
                           Almogran 12.5mg tablets                                        0.016417
Eletriptan                 Eletriptan 40mg tablets                                        0.819911
                           Eletriptan 20mg tablets                                        0.149888
                           Relpax 40mg tablets                                            0.024609
                           Relpax 20mg tablets                                            0.005593
Frovatriptan               Frovatriptan 2.5mg tablets                                     0.932477
                           Migard 2.5mg tablets                                           0.067523
Naratriptan hydrochloride  Naratriptan 2.5mg tablets                                      0.937677
                      

## Rizatriptan

In [15]:
rizatriptan_df = df[df["drug_desc"].str.contains("Rizatriptan")]

In [16]:
rizatriptan_df[rizatriptan_df['drug_desc'].str.contains("10mg tablets")]

,year_month,practice_code,drug,drug_desc,quantity,items,total_quantity,nic,actual_cost,unidentified,normalised_nic
1255,202008,L83089,Rizatriptan,Rizatriptan 10mg tablets,12.0,1,12.0,15.08,14.03370,N,1.26
1256,202008,L83089,Rizatriptan,Rizatriptan 10mg tablets,6.0,3,18.0,22.62,21.06915,N,1.26
7735,202008,L83043,Rizatriptan,Rizatriptan 10mg tablets,9.0,1,9.0,11.31,10.52837,N,1.26
9253,202008,K84013,Rizatriptan,Rizatriptan 10mg tablets,6.0,1,6.0,7.54,7.02305,N,1.26
12545,202008,L83147,Rizatriptan,Rizatriptan 10mg tablets,12.0,3,36.0,45.24,42.10110,N,1.26
...,...,...,...,...,...,...,...,...,...,...,...
16675322,202008,L83004,Rizatriptan,Rizatriptan 10mg tablets,12.0,3,36.0,45.24,42.10110,N,1.26
16675323,202008,L83004,Rizatriptan,Rizatriptan 10mg tablets,16.0,1,16.0,20.11,18.81057,N,1.26
16675324,202008,L83004,Rizatriptan,Rizatriptan 10mg tablets,6.0,2,12.0,15.08,14.04610,N,1.26
16676507,202008,H85020,Rizatriptan,Rizatriptan 10mg tablets,12.0,2,24.0,30.16,28.06740,N,1.26


In [17]:
rizatriptan_df["drug_desc"].value_counts(normalize=True)

Rizatriptan 10mg orodispersible tablets sugar free    0.324035
Rizatriptan 10mg oral lyophilisates sugar free        0.317413
Rizatriptan 10mg tablets                              0.299521
Rizatriptan 5mg tablets                               0.058890
Rizatriptan 5mg orodispersible tablets sugar free     0.000141
Name: drug_desc, dtype: float64

In [18]:
rizatriptan_df["normalised_nic"].value_counts(normalize=True)

4.46    0.377360
1.73    0.321851
1.26    0.298535
3.09    0.000352
4.45    0.000352
1.25    0.000352
2.64    0.000282
1.74    0.000282
2.86    0.000141
2.07    0.000070
2.28    0.000070
2.34    0.000070
1.40    0.000070
1.93    0.000070
1.55    0.000070
0.96    0.000070
Name: normalised_nic, dtype: float64

### Rizatriptan formulation - normalised_nic
- **10mg tablets** - **1.26**
- **10mg oral lyophilisates** - **4.46** 
- **10mg orodispersible tablets** - **1.73**. Small number have other normalised NIC (3.09, 4.45)
- The above 3 formulations make up >93% of generic rizatriptan precriptions
- (5mg tablets - 4.46)

In [19]:
maxalt_df = df[df["drug_desc"].str.contains("Maxalt")]

In [20]:
maxalt_df['normalised_nic'].unique()

array([4.46])

In [21]:
maxalt_df['drug_desc'].unique()

array(['Maxalt 10mg tablets', 'Maxalt Melt 10mg oral lyophilisates',
       'Maxalt 5mg tablets'], dtype=object)

In [22]:
maxalt_df_tablet = maxalt_df[~maxalt_df["drug_desc"].str.contains("lyophilisates")]

In [23]:
maxalt_df_tablet["normalised_nic"].unique()

array([4.46])

In [24]:
maxalt_df_tablet["drug_desc"].value_counts()

Maxalt 10mg tablets    279
Maxalt 5mg tablets      59
Name: drug_desc, dtype: int64

**The 279 prescriptions for Maxalt 10mg are at over 3.5 times the price of the equivalent generic** (which was prescribed 4252 times this month)


In [25]:
maxalt_10_tab_df = maxalt_df_tablet[maxalt_df_tablet["drug_desc"] == "Maxalt 10mg tablets"]
maxalt_10_tab_df

,year_month,practice_code,drug,drug_desc,quantity,items,total_quantity,nic,actual_cost,unidentified,normalised_nic
5423,202008,L83043,Rizatriptan,Maxalt 10mg tablets,12.0,1,12.0,53.48,49.73780,N,4.46
77146,202008,Y04662,Rizatriptan,Maxalt 10mg tablets,12.0,1,12.0,53.48,49.73780,N,4.46
133756,202008,M85128,Rizatriptan,Maxalt 10mg tablets,12.0,1,12.0,53.48,49.73780,N,4.46
189951,202008,C82100,Rizatriptan,Maxalt 10mg tablets,12.0,1,12.0,53.48,49.73780,N,4.46
258999,202008,M85066,Rizatriptan,Maxalt 10mg tablets,12.0,2,24.0,106.96,99.47561,N,4.46
...,...,...,...,...,...,...,...,...,...,...,...
16521585,202008,M83682,Rizatriptan,Maxalt 10mg tablets,6.0,1,6.0,26.74,24.87510,N,4.46
16599078,202008,K81651,Rizatriptan,Maxalt 10mg tablets,6.0,1,6.0,26.74,24.87510,N,4.46
16610234,202008,K81027,Rizatriptan,Maxalt 10mg tablets,12.0,2,24.0,106.96,99.47561,N,4.46
16630250,202008,K82079,Rizatriptan,Maxalt 10mg tablets,12.0,3,36.0,160.44,149.20101,N,4.46


In [26]:
maxalt_10_tab_df["actual_cost"].sum()

14818.873319999999

In [27]:
maxalt_10_tab_df["total_quantity"].sum()

3575.0

In [28]:
4.46*(maxalt_10_tab_df["total_quantity"].sum())

15944.5

In [29]:
3575*1.26

4504.5

**Between ~£15,000 - 16,000 was spent on Maxalt 10mg tablets** (the error margin is due to the difference between the NIC and actual cost)

**This quantity of tablets (3575 x 10mg rizatriptan) would have cost ~£4,500 had the generic version been prescribed/dispensed. That's > £10,000 saving from 279 prescriptions of this uncommon drug in one month.**

The practice codes are below. Repeating monthly over a year or more might highlight some patterns in the practices unnecessarily prescribing branded versions of certain drugs, allowing targeted feedback. 

Even better would be a system which flags unnecssary branded prescriptions. This would ideally be at the time of prescription, which given many/most/basically all(?) prescriptions are electronic these days, I'm not sure why this isn't happening. When the GP makes the prescription, a quick box saying "there is a equivalent generic version, do you agree to this switch?" seems logical.

In [30]:
maxalt_10_tab_df["practice_code"].value_counts()

P91012    3
Y00351    2
F81212    2
P81017    2
N82062    2
         ..
P84030    1
E85130    1
C82100    1
M85028    1
C87012    1
Name: practice_code, Length: 268, dtype: int64

## Frovatriptan

In [31]:
frova_df = df[df["drug_desc"].str.contains("Frovatriptan")]
frova_df

,year_month,practice_code,drug,drug_desc,quantity,items,total_quantity,nic,actual_cost,unidentified,normalised_nic
21969,202008,N84034,Frovatriptan,Frovatriptan 2.5mg tablets,6.0,1,6.0,6.11,5.69344,N,1.02
28165,202008,K84054,Frovatriptan,Frovatriptan 2.5mg tablets,12.0,5,60.0,61.10,56.87244,N,1.02
32707,202008,N84028,Frovatriptan,Frovatriptan 2.5mg tablets,6.0,1,6.0,6.11,5.69344,N,1.02
42488,202008,J83037,Frovatriptan,Frovatriptan 2.5mg tablets,12.0,1,12.0,12.22,11.37449,N,1.02
47381,202008,J83008,Frovatriptan,Frovatriptan 2.5mg tablets,12.0,2,24.0,24.44,22.74898,N,1.02
...,...,...,...,...,...,...,...,...,...,...,...
16672324,202008,H85020,Frovatriptan,Frovatriptan 2.5mg tablets,6.0,1,6.0,6.11,5.69344,N,1.02
16672325,202008,H85020,Frovatriptan,Frovatriptan 2.5mg tablets,12.0,2,24.0,24.44,22.74898,N,1.02
16672326,202008,H85020,Frovatriptan,Frovatriptan 2.5mg tablets,24.0,1,24.0,24.44,22.73658,N,1.02
16672787,202008,L83004,Frovatriptan,Frovatriptan 2.5mg tablets,6.0,2,12.0,12.22,11.38689,N,1.02


In [38]:
frova_df["normalised_nic"].value_counts(normalize = True)

1.02    0.995761
2.64    0.002119
1.83    0.001060
1.34    0.000353
1.56    0.000353
1.29    0.000353
Name: normalised_nic, dtype: float64

In [32]:
migard_df = df[df["drug_desc"] == "Migard 2.5mg tablets"]
migard_df

,year_month,practice_code,drug,drug_desc,quantity,items,total_quantity,nic,actual_cost,unidentified,normalised_nic
61973,202008,L83096,Frovatriptan,Migard 2.5mg tablets,12.0,2,24.0,66.68,62.02349,N,2.78
85206,202008,L83027,Frovatriptan,Migard 2.5mg tablets,18.0,1,18.0,50.01,46.51142,N,2.78
154203,202008,M85079,Frovatriptan,Migard 2.5mg tablets,24.0,1,24.0,66.68,62.01109,N,2.78
226027,202008,A82053,Frovatriptan,Migard 2.5mg tablets,6.0,2,12.0,33.34,31.02415,N,2.78
350130,202008,P82640,Frovatriptan,Migard 2.5mg tablets,6.0,1,6.0,16.67,15.51207,N,2.78
...,...,...,...,...,...,...,...,...,...,...,...
16315753,202008,M83129,Frovatriptan,Migard 2.5mg tablets,6.0,1,6.0,16.67,15.51207,N,2.78
16325603,202008,M85078,Frovatriptan,Migard 2.5mg tablets,12.0,1,12.0,33.34,31.01175,N,2.78
16456940,202008,K81069,Frovatriptan,Migard 2.5mg tablets,12.0,1,12.0,33.34,31.01175,N,2.78
16629402,202008,G84023,Frovatriptan,Migard 2.5mg tablets,24.0,1,24.0,66.68,62.01109,N,2.78


In [39]:
migard_df["normalised_nic"].value_counts(normalize=True)

2.78    1.0
Name: normalised_nic, dtype: float64

In [40]:
migard_df["actual_cost"].sum()

6864.08722

In [43]:
migard_df["total_quantity"].sum()

2656.0

In [46]:
1.02*(migard_df["total_quantity"].sum())

2709.12

### Prescribing generic rather than branded version of Frovatriptan could have saved ~£4,000 on 205 prescriptions across England in August 2020

In [33]:
migard_df["practice_code"].value_counts()

B81020    2
M82023    2
B81675    2
D82006    2
M82013    2
         ..
M81035    1
J82018    1
M85779    1
P81032    1
B81040    1
Name: practice_code, Length: 194, dtype: int64

### Further steps

#### Generalise method to all drugs and automate to include past and future months

1) **Group all drugs by scientific drug name, then dose, then formulation, and finally whether branded or generic.** This will allow comparison of the exact same medication clinically (for most drugs in which the brand is not clinically important) for cost (actual_cost or normalised_nic)

2) Plot/compare cost for branded and generic versions of the same drugs
- **Calculate difference in normalised_nic between drugs that have been grouped by scientific name, dose & formulation** (which would leave groups of the same medication, except for the branding)
- **Sort/plot by normalised_nic_diff to see which drugs have the biggest different in cost between generic & branded version**
- **Function of normalised_nic_diff * total_quantity of branded prescription is the definition of drugs liable for the greatest savings**
- Starting at the top, could evaluate which drugs had evidence-based indications for prescribing branded versions over generic (e.g. some AEDs, renal drugs etc.): [OpenPrescribing](https://openprescribing.net/national/england/) national page provides a high-level overview of target areas (E.g. ARBs)

3) Assess which practices are most commonly prescribing the more expensive version of the drugs over 3/6/etc. month (rolling) periods